# Boundary Detection

```{note}
This post extends concepts discussed in [a previous post about pixels as vectors](images).
You don't *need* to read that one first.
```

## Overview

There are plenty of tools available for performing image processing,
but let's say that we want to implement our own *boundary detection* algorithm.
How would you go about that?
Here is my completely naive attempt at trying to solve this problem.

Now, I'm not an image processing wizard, but my guess is that you would
somehow need to compare pixel value(s) to their neighbour(s).
A simple solution could look like this:

* If pixel values are similar enough, then the pixels might belong to the same object.
* If pixel values are not similar enough, then they belong to different objects.

This seems simple enough, but in order to really
solve this problem we will need to specify a few more things:

* How do we determine *neighbors* of pixels?
* What is our method for *comparing* pixel values?
* How do we determine if pixels are *similar*?
* What constitutes an *object*?

## Finding Neighbouring Pixels

We will use a simple approach of just itereating over the rows and columns
of our images. Pixels are adjacent to eachother if they are in the same
row/column and if they are adjacent each other.


## Comparing Pixel Values

Imagine if we treat all the pixels in an image as vectors.
Vectors have both _magnitude_ and _direction_.
The [dot product](https://en.wikipedia.org/wiki/Dot_product)
is a mathematical operation that returns the angle between two unit vectors.
Could we use this simple measurement as a means for comparing pixel values?
Another metric that we could use is the magnitude of a vector.
We could probably use this metric as well to compare the brightness
of two pixels.

## Threshold 

The *dot product* of two vectors returns a single value (i.e. a *float*).
The dot product is a special number in that it returns the angle between
two vectors.
Could our logic be as simple as computing the dot product of two pixels,
and it is below some threshold, then we can assume the pixels belong to the same object?

## What Constitutes an Object

We are representing an object as a collection of pixels, right?
If multiple pixels are connected, that's an object.
So, lone pixels not connected to any other pixels are not objects.
Moreover, we may want to also specify

## Representing Connected Pixels

We will use one of my favourite techniques: [graph processing with networkx](https://networkx.org/documentation/stable/index.html)!
We will use *networkx* to connect neighboring pixels.
If we determine that pixels belong to the same object
then we create a connection between the pixels.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import networkx as nx

In [ ]:
img_url = "https://github.com/scikit-image/scikit-image/blob/main/skimage/data/phantom.png?raw=true"
img_url = "https://github.com/scikit-image/scikit-image/blob/main/skimage/data/astronaut.png?raw=true"
img = mpimg.imread(img_url)
# img = img[::5,::5,:]
plt.imshow(img)
plt.show()

In [ ]:
def create_graph(img: np.ndarray) -> nx.Graph:
    nrows, ncols, _ = img.shape
    g = nx.Graph(nrows=nrows, ncols=ncols, img=img)
    for r in range(nrows):
        for c in range(ncols):
            g.add_node((r, c), color=img[r, c], group=None)

    return g


def draw_graph(g, node_size=1):
    pos = {n: (n[1], -n[0]) for n in g.nodes}
    node_color = [g.nodes[n]["group"] for n in g.nodes]
    if any(nc is None for nc in node_color):
        node_color = [g.nodes[n]["color"] for n in g.nodes]
    nx.draw_networkx(
        g,
        pos=pos,
        node_size=node_size,
        node_color=node_color,
        with_labels=False,
    )


g = create_graph(img)
draw_graph(g, node_size=10)

In [ ]:
def compute_vector_metrics(v):
    mag = np.linalg.norm(v)
    if mag == 0.0:
        return v, mag
    unit = v / mag
    return unit, mag


def are_pixels_connected(
    p1, p2, mag_threshold: float, dot_threshold: float
) -> bool:
    p1u, p1m = compute_vector_metrics(p1)
    p2u, p2m = compute_vector_metrics(p2)

    dot = np.dot(p1u, p2u)

    if abs(p1m - p2m) <= mag_threshold:
        if abs(dot) <= dot_threshold:
            return True
        
    return False

def pixel_iterator(g):
    nrows = g.graph["nrows"]
    ncols = g.graph["ncols"]
    for r in range(nrows):
        for c in range(ncols-1):
            yield (r,c), (r,c+1)
    for r in range(nrows-1):
        for c in range(ncols):
            yield (r,c), (r+1,c)
            

def determine_connected_pixels(g: nx.Graph, object_size_threshold, **kwargs):
    g = g.copy()

    for s, t in pixel_iterator(g):
        ps = g.nodes[s]["color"]
        pt = g.nodes[t]["color"]
        if are_pixels_connected(ps, pt, **kwargs):
            g.add_edge(s, t)

    gc = nx.connected_components(g)
    nodes_to_remove = []
    for i, nodes in enumerate(gc):
        if len(nodes) < object_size_threshold:
            nodes_to_remove.extend(nodes)
        else:
            for n in nodes:
                g.nodes[n]["group"] = i
    g.remove_nodes_from(nodes_to_remove)
    return g

In [ ]:
g_conn = determine_connected_pixels(
    g, 1, dot_threshold=np.inf, mag_threshold=np.inf
)
print(nx.number_connected_components(g_conn))
draw_graph(g_conn, node_size=5)

In [ ]:
g_conn = determine_connected_pixels(
    g, 5, dot_threshold=0.8, mag_threshold=0.5
)
print(nx.number_connected_components(g_conn))
draw_graph(g_conn, node_size=5)

In [ ]:
g_conn = determine_connected_pixels(
    g, dot_threshold=0.01, mag_threshold=0.05
)
print(nx.number_connected_components(g_conn))
draw_graph(g_conn, node_size=5)